In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [16]:
model = load_model("model.h5")

with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encoder_geo.pkl","rb") as file:
    onehot_encoder_geo = pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler = pickle.load(file)

In [17]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [18]:
input_df = pd.DataFrame([input_data])

In [19]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [20]:
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])

In [21]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [23]:
geo_encoded = onehot_encoder_geo.transform([input_df["Geography"]]).toarray()

D:\python\python\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [24]:
geo_encoded

array([[1., 0., 0.]])

In [26]:
geo_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

In [28]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [29]:
pd.concat((input_df.drop(["Geography"],axis = 1),geo_df),axis = 1)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [30]:
input_df = pd.concat((input_df.drop(["Geography"],axis = 1),geo_df),axis = 1)

In [31]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [32]:
scaler.transform(input_df)

array([[-0.52111599,  0.91601335,  0.10961719, -0.68538967, -0.2569057 ,
         0.8095029 ,  0.64259497,  0.9687384 , -0.87203322,  0.98560362,
        -0.5698444 , -0.57369368]])

In [33]:
scaled_input = scaler.transform(input_df)

In [34]:
model.predict(scaled_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step


array([[0.0343769]], dtype=float32)

In [36]:
prediction = model.predict(scaled_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step


In [37]:
prediction

array([[0.0343769]], dtype=float32)

In [38]:
prediction_prob = prediction[0][0] 

In [39]:
prediction_prob

np.float32(0.0343769)

In [40]:
if prediction_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
